# 네이버 뉴스 API 연결해서 뉴스 스크래핑
[참고 자료]
* https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EA%B2%80%EC%83%89-api-%EB%B8%94%EB%A1%9C%EA%B7%B8-%EA%B2%80%EC%83%89-%EA%B5%AC%ED%98%84-%EC%98%88%EC%A0%9C
* https://contents.premium.naver.com/chatgpt/buff/contents/231204223117640hg


In [1]:
MY_CLIENT_ID = "hZm0ny7bv69wAHcdYffq"
MY_CLIENT_SECRET = "pIyAPPtzD2"

In [25]:
import urllib.request
import pandas as pd
import json
import re
import html

# 네이버 API의 클라이언트 ID와 시크릿 키 설정
CLIENT_ID = MY_CLIENT_ID  # 실제 클라이언트 ID를 입력하세요
CLIENT_SECRET = MY_CLIENT_SECRET  # 실제 클라이언트 시크릿을 입력하세요

# 검색할 키워드 목록
SEARCH_QUERIES = ["주식", "채권", "금융"]

# API 호출 시 설정
DISPLAY_COUNT = 100  # 한 페이지에 표시할 뉴스 항목 수
START_INDEX = 1  # 검색 결과의 시작 인덱스
END_INDEX = 300  # 검색 결과의 종료 인덱스
SORT_ORDER = "sim"  # 정렬 방식: "sim" (유사도) 또는 "date" (날짜)

# 뉴스 데이터를 저장할 데이터프레임 초기화
news_df = pd.DataFrame(columns=["Category", "Title", "Original Link", "Publication Date"])

def fetch_news(query):
    """
    주어진 검색어에 대해 뉴스 데이터를 가져와서 데이터프레임에 추가합니다.
    
    :param query: 검색어
    """
    global news_df
    encoded_query = urllib.parse.quote(query)  # 검색어를 URL 인코딩
    current_index = len(news_df)  # 현재 데이터프레임의 인덱스
    
    for start_index in range(START_INDEX, END_INDEX, DISPLAY_COUNT):
        url = f"https://openapi.naver.com/v1/search/news?query={encoded_query}&display={DISPLAY_COUNT}&start={start_index}&sort={SORT_ORDER}"
        
        try:
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id", CLIENT_ID)
            request.add_header("X-Naver-Client-Secret", CLIENT_SECRET)
            response = urllib.request.urlopen(request)
            response_code = response.getcode()
            
            if response_code == 200:
                response_body = response.read()
                response_dict = json.loads(response_body.decode('utf-8'))
                items = response_dict['items']
                
                for item in items:
                    # HTML 태그 제거
                    clean_title = re.sub(re.compile('<.*?>'), '', item['title'])
                    clean_pub_date = re.sub(re.compile('<.*?>'), '', item['pubDate'])
                    
                    # 데이터프레임에 뉴스 항목 추가
                    news_df.loc[current_index] = [
                        query,  # 카테고리 추가
                        html.unescape(clean_title),  # HTML 엔티티 제거
                        html.unescape(item['originallink']),  # HTML 엔티티 제거
                        clean_pub_date
                    ]
                    current_index += 1
            else:
                print(f"Error Code: {response_code}")
                error_details = response.read().decode('utf-8')
                print(f"Error Details: {error_details}")

        except urllib.error.HTTPError as e:
            print(f"HTTPError: {e.code} - {e.reason}")
            error_details = e.read().decode('utf-8')
            print(f"Error Details: {error_details}")

# 각 검색어에 대해 뉴스 데이터 가져오기
for query in SEARCH_QUERIES:
    fetch_news(query)

# 데이터프레임을 CSV 파일로 저장
news_df.to_csv('news_data.csv', index=False)
print("CSV 파일이 'news_data.csv'로 저장되었습니다.")

# 데이터프레임 확인
news_df

CSV 파일이 'news_data.csv'로 저장되었습니다.


,Category,Title,Original Link,Publication Date
0,주식,'부동산보단 주식'…투자 전환 강조한 김병환 금융위원장,https://www.hankyung.com/article/2024081288721,"Mon, 12 Aug 2024 17:51:00 +0900"
1,주식,"국민연금 ""핀란드 주식투자 배당 세금 96억원 환급 소송 승소""",https://www.yna.co.kr/view/AKR2024081205530053...,"Mon, 12 Aug 2024 10:43:00 +0900"
2,주식,"토스증권, ‘주식모으기’ 서비스 수수료 무료화 선언",https://view.asiae.co.kr/article/2024081209554...,"Mon, 12 Aug 2024 09:55:00 +0900"
3,주식,"토스증권 ""주식모으기 이제 무료로 이용하세요""",https://www.dt.co.kr/contents.html?article_no=...,"Mon, 12 Aug 2024 09:26:00 +0900"
4,주식,"토스證, '주식모으기' 서비스 거래 수수료 안 받는다",https://biz.sbs.co.kr/article_hub/20000186523?...,"Mon, 12 Aug 2024 09:54:00 +0900"
...,...,...,...,...
895,금융,"농협상호금융, ‘쌀맛 나는 하루’ 캠페인 전개",https://www.asiatoday.co.kr/view.php?key=20240...,"Mon, 12 Aug 2024 17:44:00 +0900"
896,금융,"대규모 부정대출 우리금융, 이제 와 '무관용 원칙' 공표",https://www.kbanker.co.kr/news/articleView.htm...,"Mon, 12 Aug 2024 14:12:00 +0900"
897,금융,"하나금융, 어르신 보양식 나눔 봉사활동 실시",http://www.babytimes.co.kr/news/articleView.ht...,"Mon, 12 Aug 2024 09:34:00 +0900"
898,금융,"하나금융, 혹서기 취약한 어르신들 위한 봉사활동 진행",http://www.thefirstmedia.net/news/articleView....,"Mon, 12 Aug 2024 15:44:00 +0900"


In [26]:
# 카테고리별 데이터 개수 확인
category_counts = news_df.groupby("Category").size()
print("카테고리별 뉴스 데이터 개수:")
print(category_counts)

카테고리별 뉴스 데이터 개수:
Category
금융    300
주식    300
채권    300
dtype: int64


In [31]:
news_df['Title'][0]

"'부동산보단 주식'…투자 전환 강조한 김병환 금융위원장"

# 뉴스 링크에 접근해서 원문 스크래핑 (실패)
p 태그인건 다 갖고오려고 했는데, 뭔가 뉴스 링크 들어가면 뉴스가 아니라 다른게 뜨는 듯...

In [11]:
import requests
from bs4 import BeautifulSoup

# 뉴스 내용 추출 함수
def extract_news_content(url):
    """
    주어진 URL에서 뉴스 내용을 추출합니다.
    :param url: 뉴스 URL
    :return: 추출된 뉴스 내용
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    content = ' '.join([p.text for p in paragraphs])
    return content[:1000]  # 첫 1000자만 반환

In [12]:
import pandas as pd

# CSV 파일에서 데이터프레임으로 읽어오기
news_df = pd.read_csv('news_data.csv')

# 데이터프레임 확인
news_df['Original Link'][0]

'https://www.hankyung.com/article/2024081288721'

In [13]:
# 뉴스 추출해보기
extract_news_content(news_df['Original Link'][0])

'한국경제 회원이 되어 보세요 지금 바로 한국경제 회원으로 가입하시고, 독점 혜택을 누려보세요 \n                        이미 회원이시면 로그인을 클릭해 주세요\n                      계정관리 마이뉴스 기자 구독 관리 마이증권 내 포트폴리오 관리 ⓒ 한경닷컴, 무단전재 및 재배포 금지 \n\n                                                                요양원 규제는 그대로…반쪽짜리 완화 비판도\n                                                            \n \n                                                            앞으로 보험회사가 방문 요양 서비스 시장에 직접 진출할 수 있게 된다. 지금까지는 자회사를 통해 제한적으로만 허용됐다. 다만 요양원 등 요양시설 사업은 여전히 진입 규제에 막혀 있어 일각에서는 ‘반쪽짜리&...\n                                                         \n\n                                                                "여행상품 구매자도 책임 있다"…커지는 티메프 환불 갈등\n                                                            \n \n                                                            티몬·위메프(티메프)의 여행상품과 상품권 환불 책임을 둘러싼 갈등이 커지고 있다. 환불금을 놓고 카드사와 전자지급결제대행(PG)사, 여행사 간 갈등이 불거진 가운데 소비자도 계약 상대방으로서 일정 부분 ...\n                                                         \n\n                    

# LangChain WebBaseLoader 활용
한줄기의 희망!!! 전체 문서를 다 가져와줌 <br>
이걸 이용해서 개행문자 기준으로 나눴을 때, 글자 길이가 길면 본문으로 간주 <br>
글자 길이 : 300자 <br>
(100자는 옆에 뜨는 광고도 다 가져오고, 너무 크면 가끔 문단을 나눈다고 p태그 여러개 적용해서 뉴스 기사 쓴 것들이 있어서 원문 소실됨)

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(news_df['Original Link'][0])
docs = loader.load()

docs[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.


NameError: name 'news_df' is not defined

In [20]:
def extract_long_texts(text, min_length=300):
    """
    주어진 텍스트에서 개행 문자로 분할한 후, 길이가 min_length 이상인 모든 텍스트 조각을 본문으로 추출
    :param text: 전체 뉴스 기사 텍스트
    :param min_length: 본문으로 간주할 최소 문자 수
    :return: 본문으로 간주된 텍스트 조각들
    """
    # 개행 문자(\n)를 기준으로 텍스트를 분할
    segments = text.split('\n')
    
    # 길이가 min_length 이상인 텍스트 조각을 찾음
    long_texts = [segment for segment in segments if len(segment) > min_length]
    
    # 모든 긴 텍스트 조각을 하나의 문자열로 결합
    return '\n'.join(long_texts)

text = docs[0].page_content

content_text = extract_long_texts(text)
content_text

'                김병환 금융위원장(사진)은 12일 “밸류업(기업 가치 제고) 정책이 안착하면 경제 구조가 부채 중심에서 자본 중심으로 전환될 것”이라고 말했다.그는 이날 LG, 현대자동차, 포스코와 유관기관이 참여한 가운데 연 기업 밸류업 상장기업 간담회에서 “밸류업 정책이 우리 경제의 역동성·안정성을 높이고 지속가능한 성장에 기여할 것”이라며 이같이 설명했다.김 위원장의 발언은 자산 대부분에 대출금까지 얹어 부동산에 투자하는 가계 자산 운용 구조를 바꿔야 한다는 의미로 해석된다. 부동산보다는 주식 등에 투자해 노후 자산을 불려야 한다는 것이다.김 위원장은 “정부는 다음달 코리아 밸류업 지수를 발표하고 올 4분기 이 지수를 기초자산으로 하는 상장지수펀드(ETF)를 출시하는 작업을 차질 없이 추진할 것”이라고 말했다. 최근 증시가 급등락한 것과 관련해서는 “우리 증시의 과도한 낙폭과 더딘 회복 속도에 대해 아쉬워하는 평가가 있다”며 “보다 단단하고 회복력을 갖춘 증시로 도약하려면 밸류업 확산·내실화가 필요하다”고 했다.김익환 기자 lovepen@hankyung.com'

# naver news api와 WebBaseLoader를 활용한 뉴스 원문 스크래핑 전체 코드

In [4]:
MY_CLIENT_ID = "hZm0ny7bv69wAHcdYffq"
MY_CLIENT_SECRET = "pIyAPPtzD2"

In [5]:
import urllib.request
import pandas as pd
import json
import re
import html
from langchain_community.document_loaders import WebBaseLoader

# 네이버 API의 클라이언트 ID와 시크릿 키 설정
CLIENT_ID = MY_CLIENT_ID  # 실제 클라이언트 ID를 입력하세요
CLIENT_SECRET = MY_CLIENT_SECRET  # 실제 클라이언트 시크릿을 입력하세요

# 검색할 키워드 목록
SEARCH_QUERIES = ["주식", "채권", "금융"]

# API 호출 시 설정
DISPLAY_COUNT = 100  # 한 페이지에 표시할 뉴스 항목 수
START_INDEX = 1  # 검색 결과의 시작 인덱스
END_INDEX = 300  # 검색 결과의 종료 인덱스
SORT_ORDER = "sim"  # 정렬 방식: "sim" (유사도) 또는 "date" (날짜)

# 뉴스 데이터를 저장할 데이터프레임 초기화
news_df = pd.DataFrame(columns=["Category", "Title", "Original Link", "Publication Date", "Content"])

def fetch_news(query):
    """
    주어진 검색어에 대해 뉴스 데이터를 가져와서 데이터프레임에 추가합니다.
    
    :param query: 검색어
    """
    global news_df
    encoded_query = urllib.parse.quote(query)  # 검색어를 URL 인코딩
    current_index = len(news_df)  # 현재 데이터프레임의 인덱스
    
    for start_index in range(START_INDEX, END_INDEX, DISPLAY_COUNT):
        url = f"https://openapi.naver.com/v1/search/news?query={encoded_query}&display={DISPLAY_COUNT}&start={start_index}&sort={SORT_ORDER}"
        
        try:
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id", CLIENT_ID)
            request.add_header("X-Naver-Client-Secret", CLIENT_SECRET)
            response = urllib.request.urlopen(request)
            response_code = response.getcode()
            
            if response_code == 200:
                response_body = response.read()
                response_dict = json.loads(response_body.decode('utf-8'))
                items = response_dict['items']
                
                for item in items:
                    # HTML 태그 제거
                    clean_title = re.sub(re.compile('<.*?>'), '', item['title'])
                    clean_pub_date = re.sub(re.compile('<.*?>'), '', item['pubDate'])
                    
                    # 뉴스 항목을 데이터프레임에 추가
                    news_df.loc[current_index] = [
                        query,  # 카테고리 추가
                        html.unescape(clean_title),  # HTML 엔티티 제거
                        html.unescape(item['originallink']),  # HTML 엔티티 제거
                        clean_pub_date,
                        ''  # 본문 내용은 빈 문자열로 초기화
                    ]
                    current_index += 1
            else:
                print(f"Error Code: {response_code}")
                error_details = response.read().decode('utf-8')
                print(f"Error Details: {error_details}")

        except urllib.error.HTTPError as e:
            print(f"HTTPError: {e.code} - {e.reason}")
            error_details = e.read().decode('utf-8')
            print(f"Error Details: {error_details}")

def fetch_article_content(url):
    """
    주어진 뉴스 기사 URL에서 본문을 가져옵니다.
    
    :param url: 뉴스 기사 URL
    :return: 뉴스 기사 본문
    """
    loader = WebBaseLoader(url)
    docs = loader.load()
    if docs:
        return docs[0].page_content
    return ""

def extract_long_texts(text, min_length=300):
    """
    주어진 텍스트에서 개행 문자로 분할한 후, 길이가 min_length 이상인 모든 텍스트 조각을 본문으로 추출
    
    :param text: 전체 뉴스 기사 텍스트
    :param min_length: 본문으로 간주할 최소 문자 수
    :return: 본문으로 간주된 텍스트 조각들
    """
    segments = text.split('\n')
    long_texts = [segment for segment in segments if len(segment) > min_length]
    return '\n'.join(long_texts)

def update_news_with_content():
    """
    데이터프레임에 저장된 뉴스 링크를 통해 원문을 가져와 데이터프레임을 업데이트합니다.
    """
    global news_df
    for index, row in news_df.iterrows():
        if pd.isna(row['Content']) or row['Content'] == '':
            print(f"Fetching content for article: {row['Title']}")
            content = fetch_article_content(row['Original Link'])
            if content:
                content_text = extract_long_texts(content)
                news_df.at[index, 'Content'] = content_text
            else:
                print(f"Failed to fetch content for URL: {row['Original Link']}")

# 각 검색어에 대해 뉴스 데이터 가져오기
for query in SEARCH_QUERIES:
    fetch_news(query)

# 뉴스 링크에서 원문을 가져와 데이터프레임 업데이트
update_news_with_content()

# 데이터프레임을 CSV 파일로 저장
news_df.to_csv('news_data.csv', index=False)
print("CSV 파일이 'news_data.csv'로 저장되었습니다.")


Fetching content for article: [단독] 도이치 주식 최다 수익자, 윤 대통령 고액 후원했다
Fetching content for article: [경제PICK] 주식 시세처럼...실시간 '부동산 통계 시스템'
Fetching content for article: 애물단지 전락한 물납주식…상속인 되살 때 최대 50% 할인해준다
Fetching content for article: 30대그룹 공익법인, 계열사 보유주식 늘리고 기부액 줄였다
Fetching content for article: 정부, 안 팔리는 '물납주식', 상속인 다시 사도록 규제 푼다
Fetching content for article: 수그러들지 않는 美경기침체 우려…골드만 "가능성 29%→41%로"
Fetching content for article: 정부, 물납주식 우선매수제도 완화…매각주체, 캠코→증권사로
Fetching content for article: 국유지 19곳에 청년주택 공급…투자형 물납주식 매각 증권사가 주관
Fetching content for article: “외국인 지난달 국내 주식 2.4조 순매수”..9개월 연속
Fetching content for article: 지난달 외국인 국내주식 2조5천억원 순매수
Fetching content for article: 주식 양도세 자칫 '세금폭탄' 맞는다…국세청이 알려주는 신고 꿀팁
Fetching content for article: 지난달 외국인 국내주식 2.5조 순매수…9개월째 '사자'
Fetching content for article: "금투세, 주식만큼 채권시장에도 부정적"
Fetching content for article: 정부, 노후청사 개발해 청년주택 2.2만가구 공급… 상속인 물납주식 재매입 요...
Fetching content for article: 국고 속 '노는 재산' 굴린다...국유지 위 청년주택 짓고 '물납주식' 처분↑
Fetching content for article:

SSLError: HTTPSConnectionPool(host='www.newsen.com', port=443): Max retries exceeded with url: /news_view.php?uid=202408130752322410 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))